<a href="https://colab.research.google.com/github/JaswanthMannem/CREWAI/blob/main/crewai_5_collaboration_financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain-openai

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [8]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

In [4]:
from google.colab import userdata
openai_key=userdata.get('OPENAI_API_KEY')
serp_key=userdata.get('SERP_API_KEY')

In [5]:
import os

openai_api_key = openai_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serp_key

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [9]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [13]:
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [14]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [15]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [16]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [18]:
from crewai import Process

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo",temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [19]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [20]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. They will have the expertise to use statistical modeling and machine learning to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL."}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://

In [21]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

Introduction:
The purpose of this risk analysis report is to evaluate the risks associated with the proposed trading strategies and execution plans for AAPL in light of the current market conditions. The recent slight decrease in revenue and earnings compared to the previous year has raised concerns about the potential risks involved in trading AAPL. This report aims to provide a detailed analysis of these risks and suggest mitigation strategies to minimize potential losses.

Market Analysis:
The current market conditions for AAPL indicate a slight decrease in revenue and earnings compared to the previous year. The stock price for AAPL is currently at 229.00 USD and has decreased by -0.34% in the past 24 hours. Analysts offering price forecasts for Apple have a median target of 218.23, with a high estimate of 300.00 and a low estimate of 158.00. The company's fundamentals are considered average, with room for improvement according to the Barron's analysis.

Risk Analysis:
1. Market Risk:
The market risk associated with trading AAPL is significant due to the recent decrease in revenue and earnings. The stock price fluctuations and analyst forecasts indicate a level of uncertainty in the market. To mitigate this risk, it is recommended to implement safeguards such as stop-loss orders to limit potential losses.

2. Company-Specific Risk:
AAPL's fundamentals are considered average, which poses a risk to trading strategies. Any negative news or developments related to the company could impact the stock price. To address this risk, it is advisable to diversify the portfolio and monitor the stock performance closely to make informed trading decisions.

3. Economic Risk:
The overall economic conditions can also affect the performance of AAPL. Any macroeconomic factors such as changes in interest rates, inflation, or geopolitical events can impact the stock price. To mitigate economic risks, staying informed about the latest news and developments related to AAPL is crucial.

Mitigation Strategies:
Based on the risk analysis conducted, the following mitigation strategies are recommended to minimize potential losses:
- Implement stop-loss orders to limit losses in case of adverse market conditions.
- Diversify the portfolio to spread risk across different assets and sectors.
- Monitor the stock performance closely and stay informed about the latest news and developments related to AAPL.
- Consider using technical analysis tools to identify potential entry and exit points for trading AAPL.

Conclusion:
In conclusion, the risks associated with the proposed trading strategies and execution plans for AAPL are significant due to the current market conditions. It is crucial to carefully evaluate these risks and implement the suggested mitigation strategies to minimize potential losses. By following the recommendations outlined in this report, traders can make informed decisions and navigate the market conditions effectively.